In [ ]:
from bg_net.utils import get_most_recent
from bg_net.injection_efficiency.config import config
from bg_net.archiver.indexer_utils import str_to_jst_datetime

In [ ]:
OBS_HER = "TM_EVR0_HER_INJ_EFF_BCM"
OBS_LER = "TM_EVR0_LER_INJ_EFF_BCM"
target_key = OBS_LER
# Set path to data, adjust to your system
path_to_data = "/home/lukas/archiver_data"
# Set path to trained injection efficiency model
path_to_model = "/home/lukas/bg_net/examples/serving_model/" + target_key
# Select time intervall used for trainig 
start = str_to_jst_datetime("2022-06-01 00:00:00")
end = str_to_jst_datetime("2022-06-23 23:59:59")

In [ ]:
model = get_most_recent(path_to_model)

In [ ]:
# The constraint for the baysian optimization search space in the format [{PV}, {middle}, {range around middle}]
optimization_space = [
    ["CGLINJ_SEPTUM_ANG_R", 4.107075, 6e-02],
    ["CGLINJ_SEPTUM_POS_R", 33.995190, 6e-02],
    ["CGLINJ_KICKER_1_ANGLE", 1.777718, 1e-06],
    ["CGLINJ_KICKER_2_ANGLE", 1.551764, 1e-06],
]